In [2]:
""" cross-iris.py """
from sklearn import svm, metrics
import random, re

# 붓꽃의 CSV 파일 읽어 들이기 --- (※1)
lines = open('iris.csv', 'r', encoding='utf-8').read().split("\n")
f_tonum = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
f_cols  = lambda li: list(map(f_tonum,li.strip().split(',')))
csv = list(map(f_cols, lines))
del csv[0] # 헤더 제거하기
random.shuffle(csv) # 데이터 섞기

# 데이터를 K개로 분할하기 --- (※2)
K = 5 
csvk = [ [] for i in range(K) ]
for i in range(len(csv)):
    csvk[i % K].append(csv[i])

# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하는 함수
def split_data_label(rows):
    data = []; label = []
    for row in rows:
        data.append(row[0:4])
        label.append(row[4])
    return (data, label)

# 정답률 구하기 --- (※3)
def calc_score(test, train):
    test_f, test_l = split_data_label(test)
    train_f, train_l = split_data_label(train)
    # 학습시키고 정답률 구하기
    clf = svm.SVC()
    clf.fit(train_f, train_l)
    pre = clf.predict(test_f)
    return metrics.accuracy_score(test_l, pre)

# K개로 분할해서 정답률 구하기 --- (※4)
score_list = []
for testc in csvk:
    # testc 이외의 데이터를 훈련 전용 데이터로 사용하기
    trainc = []
    for i in csvk:
        if i != testc: trainc += i
    sc = calc_score(testc, trainc)
    score_list.append(sc)

print("각각의 정답률 =", score_list)
print("평균 정답률 =", sum(score_list) / len(score_list))

각각의 정답률 = [0.9666666666666667, 0.8666666666666667, 0.9666666666666667, 1.0, 1.0]
평균 정답률 = 0.9600000000000002


In [3]:
""" cross-iris2.py """
import pandas as pd
from sklearn import svm, metrics, model_selection
import random, re

# 붓꽃의 CSV 데이터 읽어 들이기 --- (※1)
csv = pd.read_csv('iris.csv')

# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하기 --- (※2)
data = csv[["SepalLength","SepalWidth","PetalLength","PetalWidth"]]
label = csv["Name"]

# 크로스 밸리데이션하기 --- (※3)
clf = svm.SVC()
scores = model_selection.cross_val_score(clf, data, label, cv=5)
print("각각의 정답률 =", scores)
print("평균 정답률 =", scores.mean())

각각의 정답률 = [0.96666667 0.96666667 0.96666667 0.93333333 1.        ]
평균 정답률 = 0.9666666666666666


In [26]:
""" grid-mnist.py """
import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV

# MNIST 학습 데이터 읽어 들이기 --- (※1)
train_csv = pd.read_csv("./mnist/train.csv")
test_csv  = pd.read_csv("./mnist/t10k.csv")

# 필요한 열 추출하기 --- (※2)
"""
*csv.loc[]를 사용하면 KeyError가 나타남. 중간에 빈 row가 있어서 그런 것으로 추정
*csv.iloc[]를 사용해 문제를 해결했으나, scikit-learn version 1.2 이상에서 사용 불가능하다고 함
cf. https://pandas.pydata.org/pandas-docs/version/0.15.0/indexing.html
"""
train_label = train_csv.iloc[:,0]
train_data  = train_csv.iloc[:,1:577]
test_label  = test_csv.iloc[:,0]
test_data   = test_csv.iloc[:,1:577]
print("학습 데이터의 수 =", len(train_label))

# 그리드 서치 매개변수 후보 설정 --- (※3)
params = [
    {"C": [1,10,100,1000], "kernel":["linear"]},
    {"C": [1,10,100,1000], "kernel":["rbf"], "gamma":[0.001, 0.0001]}
]

# 그리드 서치 수행 --- (※4)
# n-jobs를 -1로 하면 코어 수에 맞게, 자동으로 병렬 계산할 프로세스 수를 정해줌
clf = GridSearchCV( svm.SVC(), params, n_jobs=-1 )
clf.fit(train_data, train_label)
print("학습기 =", clf.best_estimator_)

# 테스트 데이터 확인하기 --- (※5)
pre = clf.predict(test_data)
ac_score = metrics.accuracy_score(pre, test_label)
print("정답률 =",ac_score)

학습 데이터의 수 = 1000
학습기 = SVC(C=1, kernel='linear')
정답률 = 0.866


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0.451
- 0.452
- 0.453
- 0.454
- 0.455
- ...
Feature names seen at fit time, yet now missing:
- 1
- 1.1
- 107
- 108
- 11
- ...

  warnings.warn(message, FutureWarning)
